In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.

# Setting features based on important features from Model 1

selected_features = df.drop(columns="koi_disposition")

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, df["koi_disposition"], random_state=1)

In [7]:
X_train.shape

(5243, 40)

In [8]:
y_train.unique()

array(['CANDIDATE', 'CONFIRMED', 'FALSE POSITIVE'], dtype=object)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [9]:
# Scale your data
from sklearn.preprocessing import LabelEncoder,StandardScaler

X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_test_scaled

array([[-0.42276709, -0.57125689, -0.50369272, ..., -0.03351199,
        -1.54207582,  1.27194921],
       [ 2.36536859, -0.57125689, -0.50369272, ...,  0.01304448,
        -0.80378962,  0.68911767],
       [ 2.36536859, -0.57125689, -0.50369272, ..., -0.04804041,
         1.64326243,  0.74925698],
       ...,
       [ 2.36536859, -0.57125689, -0.50369272, ..., -0.42375104,
        -0.74210221,  0.83389748],
       [-0.42276709,  1.75052595, -0.50369272, ...,  1.04918031,
         1.55855898, -0.91756696],
       [-0.42276709, -0.57125689, -0.50369272, ..., -1.14541338,
        -1.01054017, -1.68972594]])

In [10]:
from tensorflow.keras.utils import to_categorical

In [11]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [12]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
y_train_categorical

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_shape=X_train_scaled.shape[1:]))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [14]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


# Train the Model



In [16]:
 model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
164/164 - 0s - loss: 0.4664 - accuracy: 0.8062
Epoch 2/60
164/164 - 0s - loss: 0.3022 - accuracy: 0.8720
Epoch 3/60
164/164 - 0s - loss: 0.2734 - accuracy: 0.8842
Epoch 4/60
164/164 - 0s - loss: 0.2602 - accuracy: 0.8890
Epoch 5/60
164/164 - 0s - loss: 0.2511 - accuracy: 0.8905
Epoch 6/60
164/164 - 0s - loss: 0.2454 - accuracy: 0.8949
Epoch 7/60
164/164 - 0s - loss: 0.2316 - accuracy: 0.9004
Epoch 8/60
164/164 - 0s - loss: 0.2282 - accuracy: 0.9018
Epoch 9/60
164/164 - 0s - loss: 0.2239 - accuracy: 0.9022
Epoch 10/60
164/164 - 0s - loss: 0.2154 - accuracy: 0.9031
Epoch 11/60
164/164 - 0s - loss: 0.2125 - accuracy: 0.9044
Epoch 12/60
164/164 - 0s - loss: 0.2119 - accuracy: 0.9081
Epoch 13/60
164/164 - 0s - loss: 0.2016 - accuracy: 0.9140
Epoch 14/60
164/164 - 0s - loss: 0.1976 - accuracy: 0.9107
Epoch 15/60
164/164 - 0s - loss: 0.1957 - accuracy: 0.9140
Epoch 16/60
164/164 - 0s - loss: 0.1909 - accuracy: 0.9165
Epoch 17/60
164/164 - 0s - loss: 0.1893 - accuracy: 0.9187
Epoch 

In [17]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.4131 - accuracy: 0.8879
Normal Neural Network - Loss: 0.4131062924861908, Accuracy: 0.8878718614578247


In [18]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Predicted classes: ['CONFIRMED' 'FALSE POSITIVE' 'FALSE POSITIVE' 'CONFIRMED'
 'FALSE POSITIVE']
Actual Labels: ['CONFIRMED', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'FALSE POSITIVE']


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [19]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def getModel(optimizer):
    model2 = Sequential()
    model2.add(Dense(100, activation='relu', input_shape=X_train_scaled.shape[1:]))
    model2.add(Dense(100, activation='relu'))
    model2.add(Dense(3, activation='softmax'))
    model2.compile(optimizer=optimizer , loss = "categorical_crossentropy" , metrics=['accuracy'])
    return model


optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
epochs = [10, 60, 120]

param_grid = dict(epochs=epochs, optimizer=optimizer)

Kmodel = KerasClassifier(build_fn=getModel, verbose=1)
grid = GridSearchCV(estimator=Kmodel, param_grid=param_grid)
grid_result = grid.fit(X_train_scaled, y_train_categorical)

Epoch 1/10
132/132 [==============================] - 0s 453us/step - loss: 0.0889 - accuracy: 0.9678
Epoch 2/10
132/132 [==============================] - 0s 438us/step - loss: 0.0880 - accuracy: 0.9671
Epoch 3/10
132/132 [==============================] - 0s 431us/step - loss: 0.0909 - accuracy: 0.9669
Epoch 4/10
132/132 [==============================] - 0s 431us/step - loss: 0.0902 - accuracy: 0.9638
Epoch 5/10
132/132 [==============================] - 0s 453us/step - loss: 0.0842 - accuracy: 0.9723
Epoch 6/10
132/132 [==============================] - 0s 446us/step - loss: 0.0906 - accuracy: 0.9671
Epoch 7/10
132/132 [==============================] - 0s 461us/step - loss: 0.0779 - accuracy: 0.9719
Epoch 8/10
132/132 [==============================] - 0s 446us/step - loss: 0.0807 - accuracy: 0.9700
Epoch 9/10
132/132 [==============================] - 0s 446us/step - loss: 0.0778 - accuracy: 0.9719
Epoch 10/10
33/33 [==============================] - 0s 603us/step - loss: 0.1224 

33/33 [==============================] - 0s 574us/step - loss: 0.0470 - accuracy: 0.9838
Epoch 1/10
132/132 [==============================] - 0s 446us/step - loss: 0.0399 - accuracy: 0.9876
Epoch 2/10
132/132 [==============================] - 0s 438us/step - loss: 0.0552 - accuracy: 0.9809
Epoch 3/10
132/132 [==============================] - 0s 453us/step - loss: 0.0372 - accuracy: 0.9857
Epoch 4/10
132/132 [==============================] - 0s 438us/step - loss: 0.0302 - accuracy: 0.9917
Epoch 5/10
132/132 [==============================] - 0s 446us/step - loss: 0.0281 - accuracy: 0.9924
Epoch 6/10
132/132 [==============================] - 0s 446us/step - loss: 0.0275 - accuracy: 0.9912
Epoch 7/10
132/132 [==============================] - 0s 431us/step - loss: 0.0267 - accuracy: 0.9924
Epoch 8/10
132/132 [==============================] - 0s 431us/step - loss: 0.0751 - accuracy: 0.9733
Epoch 9/10
132/132 [==============================] - 0s 431us/step - loss: 0.0346 - accuracy: 

Epoch 4/10
132/132 [==============================] - 0s 438us/step - loss: 0.0970 - accuracy: 0.9771
Epoch 5/10
132/132 [==============================] - 0s 453us/step - loss: 0.0184 - accuracy: 0.9952
Epoch 6/10
132/132 [==============================] - 0s 431us/step - loss: 0.0084 - accuracy: 0.9993
Epoch 7/10
132/132 [==============================] - 0s 431us/step - loss: 0.0069 - accuracy: 0.9998
Epoch 8/10
132/132 [==============================] - 0s 431us/step - loss: 0.0062 - accuracy: 0.9995
Epoch 9/10
132/132 [==============================] - 0s 423us/step - loss: 0.0060 - accuracy: 0.9995
Epoch 10/10
33/33 [==============================] - 0s 572us/step - loss: 0.0208 - accuracy: 0.9933
Epoch 1/10
132/132 [==============================] - 0s 453us/step - loss: 0.0196 - accuracy: 0.9955
Epoch 2/10
132/132 [==============================] - 0s 461us/step - loss: 0.0415 - accuracy: 0.9936
Epoch 3/10
132/132 [==============================] - 0s 468us/step - loss: 0.0149 

132/132 [==============================] - 0s 431us/step - loss: 0.0034 - accuracy: 0.9998
Epoch 4/10
132/132 [==============================] - 0s 431us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 5/10
132/132 [==============================] - 0s 431us/step - loss: 0.0021 - accuracy: 1.0000
Epoch 6/10
132/132 [==============================] - 0s 431us/step - loss: 0.0019 - accuracy: 1.0000
Epoch 7/10
132/132 [==============================] - 0s 438us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 8/10
132/132 [==============================] - 0s 431us/step - loss: 0.0017 - accuracy: 1.0000
Epoch 9/10
132/132 [==============================] - 0s 423us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 10/10
33/33 [==============================] - 0s 544us/step - loss: 0.0053 - accuracy: 0.9981
Epoch 1/10
132/132 [==============================] - 0s 454us/step - loss: 0.0083 - accuracy: 0.9974
Epoch 2/10
132/132 [==============================] - 0s 431us/step - loss: 0.0588 - accuracy:

132/132 [==============================] - 0s 438us/step - loss: 0.0028 - accuracy: 1.0000
Epoch 5/10
132/132 [==============================] - 0s 468us/step - loss: 0.0022 - accuracy: 1.0000
Epoch 6/10
132/132 [==============================] - 0s 468us/step - loss: 0.0019 - accuracy: 1.0000
Epoch 7/10
132/132 [==============================] - 0s 476us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 8/10
132/132 [==============================] - 0s 484us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 9/10
132/132 [==============================] - 0s 514us/step - loss: 0.0015 - accuracy: 1.0000
Epoch 10/10
33/33 [==============================] - 0s 605us/step - loss: 0.0082 - accuracy: 0.9971
Epoch 1/10
132/132 [==============================] - 0s 461us/step - loss: 0.0038 - accuracy: 0.9995
Epoch 2/10
132/132 [==============================] - 0s 468us/step - loss: 0.0020 - accuracy: 1.0000
Epoch 3/10
132/132 [==============================] - 0s 514us/step - loss: 0.0015 - accuracy:

...

132/132 [==============================] - 0s 461us/step - loss: 1.6433e-05 - accuracy: 1.0000
Epoch 11/120
132/132 [==============================] - 0s 461us/step - loss: 1.5005e-05 - accuracy: 1.0000
Epoch 12/120
132/132 [==============================] - 0s 453us/step - loss: 1.3868e-05 - accuracy: 1.0000
Epoch 13/120
132/132 [==============================] - 0s 446us/step - loss: 1.2621e-05 - accuracy: 1.0000
Epoch 14/120
132/132 [==============================] - 0s 446us/step - loss: 1.1536e-05 - accuracy: 1.0000
Epoch 15/120
132/132 [==============================] - 0s 446us/step - loss: 1.0690e-05 - accuracy: 1.0000
Epoch 16/120
132/132 [==============================] - 0s 446us/step - loss: 1.0008e-05 - accuracy: 1.0000
Epoch 17/120
132/132 [==============================] - 0s 453us/step - loss: 9.4160e-06 - accuracy: 1.0000
Epoch 18/120
132/132 [==============================] - 0s 880us/step - loss: 8.9077e-06 - accuracy: 1.0000
Epoch 19/120
132/132 [===================

Epoch 86/120
132/132 [==============================] - 0s 453us/step - loss: 4.4063e-07 - accuracy: 1.0000
Epoch 87/120
132/132 [==============================] - 0s 461us/step - loss: 4.2270e-07 - accuracy: 1.0000
Epoch 88/120
132/132 [==============================] - 0s 461us/step - loss: 4.0519e-07 - accuracy: 1.0000
Epoch 89/120
132/132 [==============================] - 0s 468us/step - loss: 3.8755e-07 - accuracy: 1.0000
Epoch 90/120
132/132 [==============================] - 0s 446us/step - loss: 3.7109e-07 - accuracy: 1.0000
Epoch 91/120
132/132 [==============================] - 0s 446us/step - loss: 3.5731e-07 - accuracy: 1.0000
Epoch 92/120
132/132 [==============================] - 0s 453us/step - loss: 3.4069e-07 - accuracy: 1.0000
Epoch 93/120
132/132 [==============================] - 0s 453us/step - loss: 3.2750e-07 - accuracy: 1.0000
Epoch 94/120
132/132 [==============================] - 0s 453us/step - loss: 3.1398e-07 - accuracy: 1.0000
Epoch 95/120
132/132 [======

132/132 [==============================] - 0s 461us/step - loss: 2.2669e-06 - accuracy: 1.0000
Epoch 42/120
132/132 [==============================] - 0s 461us/step - loss: 2.1890e-06 - accuracy: 1.0000
Epoch 43/120
132/132 [==============================] - 0s 468us/step - loss: 2.0942e-06 - accuracy: 1.0000
Epoch 44/120
132/132 [==============================] - 0s 461us/step - loss: 2.0215e-06 - accuracy: 1.0000
Epoch 45/120
132/132 [==============================] - 0s 453us/step - loss: 1.9428e-06 - accuracy: 1.0000
Epoch 46/120
132/132 [==============================] - 0s 461us/step - loss: 1.8533e-06 - accuracy: 1.0000
Epoch 47/120
132/132 [==============================] - 0s 476us/step - loss: 1.7832e-06 - accuracy: 1.0000
Epoch 48/120
132/132 [==============================] - 0s 453us/step - loss: 1.7143e-06 - accuracy: 1.0000
Epoch 49/120
132/132 [==============================] - 0s 438us/step - loss: 1.6427e-06 - accuracy: 1.0000
Epoch 50/120
132/132 [===================

132/132 [==============================] - 0s 582us/step - loss: 7.8147e-08 - accuracy: 1.0000
Epoch 117/120
132/132 [==============================] - 0s 574us/step - loss: 7.4879e-08 - accuracy: 1.0000
Epoch 118/120
132/132 [==============================] - 0s 529us/step - loss: 7.0730e-08 - accuracy: 1.0000
Epoch 119/120
132/132 [==============================] - 0s 597us/step - loss: 6.7348e-08 - accuracy: 1.0000
Epoch 120/120
33/33 [==============================] - 0s 1ms/step - loss: 0.0157 - accuracy: 0.9971
Epoch 1/120
164/164 [==============================] - 0s 639us/step - loss: 0.1235 - accuracy: 0.9863
Epoch 2/120
164/164 [==============================] - 0s 535us/step - loss: 0.0372 - accuracy: 0.9935
Epoch 3/120
164/164 [==============================] - 0s 480us/step - loss: 0.0079 - accuracy: 0.9979
Epoch 4/120
164/164 [==============================] - 0s 474us/step - loss: 0.0049 - accuracy: 0.9990
Epoch 5/120
164/164 [==============================] - 0s 468us/s

164/164 [==============================] - 0s 450us/step - loss: 3.3209e-07 - accuracy: 1.0000
Epoch 72/120
164/164 [==============================] - 0s 450us/step - loss: 3.1340e-07 - accuracy: 1.0000
Epoch 73/120
164/164 [==============================] - 0s 450us/step - loss: 2.9767e-07 - accuracy: 1.0000
Epoch 74/120
164/164 [==============================] - 0s 456us/step - loss: 2.8123e-07 - accuracy: 1.0000
Epoch 75/120
164/164 [==============================] - 0s 444us/step - loss: 2.6713e-07 - accuracy: 1.0000
Epoch 76/120
164/164 [==============================] - 0s 450us/step - loss: 2.5051e-07 - accuracy: 1.0000
Epoch 77/120
164/164 [==============================] - 0s 450us/step - loss: 2.3689e-07 - accuracy: 1.0000
Epoch 78/120
164/164 [==============================] - 0s 450us/step - loss: 2.2559e-07 - accuracy: 1.0000
Epoch 79/120
164/164 [==============================] - 0s 444us/step - loss: 2.1300e-07 - accuracy: 1.0000
Epoch 80/120
164/164 [===================

In [20]:
print(grid.best_params_)
print(grid.best_score_)

{'epochs': 120, 'optimizer': 'Nadam'}
0.9988551378250122


In [22]:
# Create model and add layers
model3 = Sequential()
model3.add(Dense(units=100, activation='relu', input_shape=X_train_scaled.shape[1:]))
model3.add(Dense(units=100, activation='relu'))
model3.add(Dense(units=3, activation='softmax'))

# Compile and fit the model
model3.compile(optimizer='Adamax',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model3.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=120,
    shuffle=True,
    verbose=2
)
    
model3_loss, model3_accuracy = model3.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model3_loss}, Accuracy: {model3_accuracy}")

Epoch 1/120
164/164 - 0s - loss: 0.5764 - accuracy: 0.7658
Epoch 2/120
164/164 - 0s - loss: 0.3627 - accuracy: 0.8535
Epoch 3/120
164/164 - 0s - loss: 0.3222 - accuracy: 0.8652
Epoch 4/120
164/164 - 0s - loss: 0.3018 - accuracy: 0.8730
Epoch 5/120
164/164 - 0s - loss: 0.2901 - accuracy: 0.8787
Epoch 6/120
164/164 - 0s - loss: 0.2811 - accuracy: 0.8814
Epoch 7/120
164/164 - 0s - loss: 0.2755 - accuracy: 0.8808
Epoch 8/120
164/164 - 0s - loss: 0.2698 - accuracy: 0.8846
Epoch 9/120
164/164 - 0s - loss: 0.2662 - accuracy: 0.8867
Epoch 10/120
164/164 - 0s - loss: 0.2602 - accuracy: 0.8920
Epoch 11/120
164/164 - 0s - loss: 0.2558 - accuracy: 0.8911
Epoch 12/120
164/164 - 0s - loss: 0.2518 - accuracy: 0.8922
Epoch 13/120
164/164 - 0s - loss: 0.2482 - accuracy: 0.8932
Epoch 14/120
164/164 - 0s - loss: 0.2445 - accuracy: 0.8949
Epoch 15/120
164/164 - 0s - loss: 0.2402 - accuracy: 0.8980
Epoch 16/120
164/164 - 0s - loss: 0.2384 - accuracy: 0.8995
Epoch 17/120
164/164 - 0s - loss: 0.2355 - accura

# Save the Model

In [23]:
# joblib doesn't work well with keras.
# saving model with Keras's model.save method

# Model with tuned hyperparemeters produces slight improvements in accuracy and loss
model3.save("stephen_eldridge_model")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: stephen_eldridge_model\assets


In [24]:
## To load the saved model, uncomment and run:

# from tensorflow import keras
# model = keras.models.load_model('stephen_eldridge_model')
